# Offline Evaluation

This notebook guides through an offline evaluation of a trained transfuser model using either LiDAR or pseudo-LiDAR as input. If you want to conduct an online evaluation, please refer to the README for instructions.

In [ ]:
from transfuser.data import CARLA_Data
from transfuser.model import TransFuser
from transfuser.config import GlobalConfig
from tqdm import tqdm
from pathlib import Path

import torch
from torch.utils.data import DataLoader
import torch.nn.functional as F
torch.backends.cudnn.benchmark = True

torch.cuda.empty_cache()

In [ ]:
# Settings
weight_path = Path('/storage/remote/atcremers57/w0014/transfuser/model_ckpt/best_model.pth')
device = 'cuda'
batch_size = 32
use_pseudolidar = False
model_has_classification_branch = False

In [ ]:
# Config
config = GlobalConfig()
if use_pseudolidar:
    config.use_pseudolidar = True
else:
    config.use_pseudolidar = False

# Data
val_set = CARLA_Data(root=config.val_data, config=config)
dataloader_val = DataLoader(val_set, batch_size=batch_size, shuffle=False, num_workers=4, pin_memory=True)

# Model
model = TransFuser(config, device, use_classification_branch=model_has_classification_branch)
model.load_state_dict(torch.load(weight_path))

In [ ]:
class Engine(object):
    """Engine that runs training and inference."""

    def __init__(self):
        self.val_loss = []

    def validate(self):
        model.eval()

        with torch.no_grad():
            num_batches = 0
            wp_epoch = 0.

            # Validation loop
            for batch_num, data in enumerate(tqdm(dataloader_val), 0):

                # create batch and move to GPU
                fronts_in = data['fronts']
                lefts_in = data['lefts']
                rights_in = data['rights']
                rears_in = data['rears']
                lidars_in = data['lidars']
                fronts = []
                lefts = []
                rights = []
                rears = []
                lidars = []
                for i in range(config.seq_len):
                    fronts.append(fronts_in[i].to(device, dtype=torch.float32))
                    if not config.ignore_sides:
                        lefts.append(lefts_in[i].to(device, dtype=torch.float32))
                        rights.append(rights_in[i].to(device, dtype=torch.float32))
                    if not config.ignore_rear:
                        rears.append(rears_in[i].to(device, dtype=torch.float32))
                    lidars.append(lidars_in[i].to(device, dtype=torch.float32))

                # driving labels
                command = data['command'].to(device)
                gt_velocity = data['velocity'].to(device, dtype=torch.float32)
                gt_steer = data['steer'].to(device, dtype=torch.float32)
                gt_throttle = data['throttle'].to(device, dtype=torch.float32)
                gt_brake = data['brake'].to(device, dtype=torch.float32)

                # target point
                target_point = torch.stack(data['target_point'], dim=1).to(device, dtype=torch.float32)

                pred_wp = model(fronts+lefts+rights+rears, lidars, target_point, gt_velocity)

                gt_waypoints = [torch.stack(data['waypoints'][i], dim=1).to(device, dtype=torch.float32)
                                for i in range(config.seq_len, len(data['waypoints']))]
                gt_waypoints = torch.stack(gt_waypoints, dim=1).to(device, dtype=torch.float32)
                wp_epoch += float(F.l1_loss(pred_wp, gt_waypoints, reduction='none').mean())

                num_batches += 1

            wp_loss = wp_epoch / float(num_batches)
            tqdm.write(f' Wp loss: {wp_loss:3.3f}')

In [ ]:
trainer = Engine()
trainer.validate()